# 문제1

In [1]:
# import 
import pandas as pd 
from urllib.request import urlopen
import time 
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

print("selenium 1")
url = "https://www.starbucks.co.kr/index.do"
driver = webdriver.Chrome("./driver/chromedriver") 
driver.get(url)
driver.maximize_window()
# driver.set_window_size(1920, 1080) 
time.sleep(3)

# xpath 스타벅스 지도 화면으로 이동
xpath_1 = '//*[@id="gnb"]/div/nav/div/ul/li[3]/h2/a'
store_tag = driver.find_element_by_xpath(xpath_1).click()
time.sleep(2)
xpath_2 = '//*[@id="container"]/section[1]/article/div[2]/a'
map_tag = driver.find_element_by_xpath(xpath_2).click()
time.sleep(2)
xpath_3 = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a'
local_tag = driver.find_element_by_xpath(xpath_3).click()
time.sleep(2)
xpath_4 = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'
seoul_tag = driver.find_element_by_xpath(xpath_4).click()
time.sleep(2)
xpath_5 = '//*[@id="mCSB_2_container"]/ul/li[1]/a'
all_tag = driver.find_element_by_xpath(xpath_5).click()
time.sleep(2)

print("selenium 2")

# 서울에 있는 매장 갯수 확인 ( 현재 위치 근처 점들 10개 제외)
store_total = driver.find_elements_by_css_selector("#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step3 > div.result_num_wrap > span")

for i in store_total:
    score = i.text
print(f"서울에 있는 매장 갯수 : {score}")


# beautifulsoup 사용 시작
driver.minimize_window()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all("li","quickResultLstCon")

datas = [] 

print("selenium 3")

# 0~9는 현재 위치 근처에 있는 스타벅스점들이므로 제외
for content in tqdm(contents[10:int(score)+10]):
    name = content.strong.text
    address = content.p.text[0:-9]
    area = address.split(" ")[1]
    datas.append({
        "가게 이름": name.split(" ")[0]+"점", 
        "주소": address,
        "구 이름": area
    })
    
driver.quit()

df = pd.DataFrame(datas)
df.to_excel("./weeklytest_starbucks_answer.xlsx", encoding="utf-8")
print("Selenium quit!")



selenium 1
selenium 2
서울에 있는 매장 갯수 : 566
selenium 3


100%|██████████| 566/566 [00:00<00:00, 28304.08it/s]


Selenium quit!


# 문제2

In [25]:
# import 
import pandas as pd 
from urllib.request import urlopen
import time 
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

seoul_area = ['강남구','강동구','강서구','강북구','관악구','광진구','구로구','금천구','노원구','동대문구','도봉구','동작구','마포구','서대문구','성동구','서초구','성북구','송파구','영등포구','용산구','양천구','은평구','종로구','중구','중랑구']
print("selenium 1")

url = "https://www.ediya.com/"
driver = webdriver.Chrome("./driver/chromedriver") 
driver.get(url)
driver.maximize_window()
# driver.set_window_size(1920, 1080) 
time.sleep(3)

# xpath 스타벅스 지도 화면으로 이동
xpath_0 = '/html/body/header/div/div/div[2]/div[1]/ul[1]/li[2]/a'
search_tag = driver.find_element_by_xpath(xpath_0).click()
time.sleep(2)

print("selenium 2")

datas = []

for i in tqdm(seoul_area):
    driver.maximize_window()
    xpath_1 = '//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a'
    driver.find_element_by_xpath(xpath_1).click()
    time.sleep(2)
    xpath_2 = '//*[@id="keyword"]'
    driver.find_element_by_xpath(xpath_2).send_keys("서울 "+i)
    time.sleep(2)
    xpath_3 = '//*[@id="keyword_div"]/form/button'
    driver.find_element_by_xpath(xpath_3).click()
    time.sleep(2)
    driver.find_element_by_xpath(xpath_2).send_keys()

    # beautifulsoup 사용 시작
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    contents = soup.find_all("li","item")


    for content in tqdm(contents):
        name = content.dt.text
        address = content.dd.text
        area = address.split(" ")[1]
        datas.append({
            "가게 이름": name, 
            "주소": address,
            "구 이름": area
        })

driver.quit()

print("selenium 3")

print(f"이디야 서울에 있는 매장 갯수 : {len(datas)}")

df = pd.DataFrame(datas)
df.to_excel("./weeklytest_ediya_answer.xlsx", encoding="utf-8")
print("Selenium quit!")

selenium 1
selenium 2


100%|██████████| 25/25 [02:34<00:00,  6.17s/it]


selenium 3
이디야 서울에 있는 매장 갯수 : 722
Selenium quit!


# 문제3

# 1. 주소를 위도, 경도 변환하여 folium 라이브러리 사용하여 시각화
- 가게 반경 100m로 마커 표시하여 겹치는 구간 있는지 체크
- 위도, 경도 변환은 google spreadsheet geocode by awesome table 기능 사용
- 변환된 파일을 이용하여 folium.Circle를 통해 구현  

In [26]:
import folium
import pandas as pd 
 

df_ediya = pd.read_csv("./ediya_df.csv", index_col=0, encoding="utf-8")
df_starbucks = pd.read_csv("./starbucks_df.csv", index_col=0, encoding="utf-8")

# folium 

# 서울시청 위치
my_map = folium.Map(location=[37.56668,126.97843], zoom_start=10.5)

# 이디야 가게 표시
for idx, rows in df_ediya.iterrows():
    folium.Circle(
        [rows["Latitude"], rows["Longitude"]],
        radius=100,
        color="#3186cc",
        fill=True,
        fill_color="#3186cc",
    ).add_to(my_map)

# 스타벅스 가게 표시
for idx, rows in df_starbucks.iterrows():
    folium.Circle(
        [rows["Latitude"], rows["Longitude"]],
        radius=100,
        color="#cc3186",
        fill=True,
        fill_color="#cc3186",
    ).add_to(my_map)

folium.Circle(
    [37.568849, 126.983140],
    radius=1500,
        color="#cc7731",
        fill=True,
        fill_color="#cc7731",
    ).add_to(my_map)

folium.Circle(
    [37.499393, 127.033600],
    radius=1500,
        color="#cc7731",
        fill=True,
        fill_color="#cc7731",
    ).add_to(my_map)

my_map
    

#### 파란색 원이 이디야이고, 빨간색 원이 스타벅스이다. 멀리서 거시적으로 바라보면
#### 이디야는 비교적 고르게 분포해있고, 스타벅스는 여러 곳에 집중된 지점들(갈색 원)이 있다.

## 이디야가 스타벅스 주변에 입주한다고 하면 두 가게의 지역별 밀도 차이가 크지 않아야 한다. 그것을 상대적 지역 분포로 알아보기로 함

In [27]:
# 스타벅스와 이디야의 구 이름별 입주 수 알아보기
df_ediya_cnt = pd.DataFrame(df_ediya.groupby(['구 이름'])['가게 이름'].count())
# 두 가게의 지점 수가 다르므로 상대적 지역 분포를 알기 위한 정규화
df_ediya_cnt["가게 이름"] = df_ediya_cnt["가게 이름"] / df_ediya_cnt["가게 이름"].max() 
print(df_ediya_cnt)

         가게 이름
구 이름          
강남구   0.957447
강동구   0.531915
강북구   0.425532
강서구   1.000000
관악구   0.617021
광진구   0.510638
구로구   0.617021
금천구   0.510638
노원구   0.659574
도봉구   0.489362
동대문구  0.468085
동작구   0.489362
마포구   0.829787
서대문구  0.446809
서초구   0.638298
성동구   0.489362
성북구   0.680851
송파구   0.851064
양천구   0.574468
영등포구  0.872340
용산구   0.319149
은평구   0.553191
종로구   0.659574
중구    0.617021
중랑구   0.553191


In [28]:
df_starbucks_cnt = pd.DataFrame(df_starbucks.groupby(['구 이름'])['가게 이름'].count())
# 두 가게의 지점 수가 다르므로 상대적 지역 분포를 알기 위한 정규화
df_starbucks_cnt["가게 이름"] = df_starbucks_cnt["가게 이름"] / df_starbucks_cnt["가게 이름"].max() 
print(df_starbucks_cnt)


         가게 이름
구 이름          
강남구   1.000000
강동구   0.183908
강북구   0.068966
강서구   0.241379
관악구   0.126437
광진구   0.195402
구로구   0.137931
금천구   0.126437
노원구   0.149425
도봉구   0.034483
동대문구  0.103448
동작구   0.126437
마포구   0.367816
서대문구  0.229885
서초구   0.551724
성동구   0.137931
성북구   0.160920
송파구   0.390805
양천구   0.195402
영등포구  0.448276
용산구   0.241379
은평구   0.091954
종로구   0.459770
중구    0.643678
중랑구   0.091954


In [29]:
# 이디야 상대적 지역 분포 - 스타벅스 상대적 지역 분포
df_cnt = pd.DataFrame()
df_cnt["가게 이름"] = df_ediya_cnt["가게 이름"] - df_starbucks_cnt["가게 이름"]
print(df_cnt)
print(abs(df_cnt["가게 이름"]).mean())

         가게 이름
구 이름          
강남구  -0.042553
강동구   0.348007
강북구   0.356566
강서구   0.758621
관악구   0.490584
광진구   0.315236
구로구   0.479090
금천구   0.384202
노원구   0.510149
도봉구   0.454879
동대문구  0.364637
동작구   0.362925
마포구   0.461971
서대문구  0.216923
서초구   0.086574
성동구   0.351431
성북구   0.519932
송파구   0.460259
양천구   0.379066
영등포구  0.424065
용산구   0.077770
은평구   0.461237
종로구   0.199804
중구   -0.026657
중랑구   0.461237
0.35977500611396424


# 평균 지역별 밀도 차이가 35% 이상 차이난다 >>> 두 가게의 지역별 밀도 차이가 크다

In [40]:
# 스타벅스와 이디야의 지역별 입주 밀도 시각화
import json
geo_path = "./02. skorea_municipalities_geo_simple.json"
geo_str = json.load(open(geo_path, encoding="utf-8"))

# 서울시청 위치
my_map = folium.Map(location=[37.56668,126.97843], zoom_start=10.5, tiles="Stamen Toner")

my_map.choropleth(
    geo_data=geo_str,
    data=df_ediya_cnt["가게 이름"],
    columns=[df_ediya_cnt.index, df_ediya_cnt["가게 이름"]],
    fill_color="PuRd",
    key_on="feature.id",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="정규화된 이디야 서울 지역 상대 밀도",
)

my_map

In [39]:
# 서울시청 위치
my_map = folium.Map(location=[37.56668,126.97843], zoom_start=10.5, tiles="Stamen Toner")


my_map.choropleth(
    geo_data=geo_str,
    data=df_starbucks_cnt["가게 이름"],
    columns=[df_starbucks_cnt.index, df_starbucks_cnt["가게 이름"]],
    fill_color="BuGn",
    key_on="feature.id",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="정규화된 스타벅스 서울 지역 상대 밀도",
)

my_map

# 유동인구가 높고 인구 밀집도가 높은 지역들(강남구, 마포구, 영등포구, 중구, 종로구)을 제외하면, 정규화 밀도 차이는 눈에 보일 정도로 있다. 


#### 이디야 - 자사 지점들과의 거리를 계산하여 고르게 분포되도록 함
#### 스타벅스 - 수요 인구 비율에 따라 지점 밀도를 비례하여 분포되도록 함

### 그러므로, 이디야가 전략적으로 스타벅스 옆에 입주하는 전략인 것이 아닌 각자 입주 전략이 다른 것으로 해석된다.